In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')

#Для визуализации
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Для моделирования
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score

In [ ]:
#train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/recommendationsv4/train.csv')
#test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/recommendationsv4/test.csv')
#submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/recommendationsv4/sample_submission.csv')

In [ ]:
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

In [ ]:
with open('/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json', 'r') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)
data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
food_dict = pd.read_json(data_json_str)

Посмотрим на мета-словарь для items

In [ ]:
food_dict.info()

In [ ]:
food_dict.sample(2)

In [ ]:
food_dict['asin'].nunique()

#В словаре есть порядка 3,8к дубликатов, удалим их и также впоследствии проверим на их наличие трейн

In [ ]:
train['asin'].nunique()

In [ ]:
test['asin'].nunique()

Посмотрим на исходные данные, основные статистики, а также наличие пропусков в столбцах

In [ ]:
print('Размер тренировочной выборки:', train.shape)
print('train data sample:')
print('Размер тестовой выборки:', test.shape)

In [ ]:
train.info()

In [ ]:
train.sample(5)

Посмотрим на основные компоненты данных пользователей, продукты и рейтинги

In [ ]:
print('Число уникальных пользователей:', train['userid'].nunique())
print('Число уникальных товаров:', train['itemid'].nunique())

In [ ]:
#посмотрим на наиболее популярные в трейне продукты
popular_products = pd.DataFrame(train.groupby('itemid')['rating'].count())
most_popular = popular_products.sort_values('rating', ascending=False)
#most_popular.head(10)
most_popular.head(30).plot(kind = "bar")

График отображает продажи наиболее популярных продуктов. как видно ,есть топ 3 продукта, число рейтингов которых сильно отличаются от остальных

In [ ]:
#посмотрим на пользователей, который выставляли больше всего оценок
users = pd.DataFrame(train.groupby('userid')['rating'].count())
active_users = users.head(30).sort_values('rating', ascending=False)
active_users.head(30).plot(kind = "bar")

In [ ]:
train['verified'].value_counts(normalize=True)

Посмотрим на пропуски в данных:

In [ ]:
def stat_na_per_percent(data):
    print(f'{data.shape}')
    for col in data.columns:
        pct_missing = np.mean(data[col].isnull())
        print('{} - {}%'.format(col, round(pct_missing*100)))
    print("END", end = '\n\n')
stat_na_per_percent(train)
stat_na_per_percent(test)


Исходный датасет состоит из 3 идентификаторов - itemid, userid и asin(ключ словаря с описанием продуктов), 9 признаков и 2 вариаций целевой переменной rating и overall:

1.   Для признака verified сделаем onehotencoding тк он низковариативен
2.   Признаки reviewTime и unixReviewTime будем обрабатывать отдельно через datetime, выделяю временные компоненты
3.Признак vote предположительно не используем, тк поле содержит большое число пропусков. Иначе, конвертируем в int и заполним пропуски 0, так как признак показывает количественную характеристику.  


признаки reviewerName, style и image пока не трогаем
Отдельной работы потребуют признаки reviewText и summary, которые будем напр изучать через tf-idf и генерировать новые фичи

In [ ]:
train['reviewTime'] = pd.to_datetime(train['reviewTime'])

In [ ]:
oldest_review = train['reviewTime'].min()

In [ ]:
train['weights'] = (train['reviewTime'] - oldest_review).dt.days + 1
train['weights'] = (train['weights'] - train['weights'].min())/(train['weights'] - train['weights'].min()).max()
train['weights'].min(), train['weights'].max()

In [ ]:
#Конвертируем verified в int
train['verified']=train['verified'].astype('int64')

In [ ]:
train['unixReviewTime'] = pd.to_datetime(train['unixReviewTime'], unit='s')
# сколько прошло дней
train['DaysPassed'] = (pd.datetime.now() - train['unixReviewTime']).dt.days
train['DaysPassed']=(train['DaysPassed'] - train['DaysPassed'].min())/(train['DaysPassed'] - train['DaysPassed'].min()).max()

In [ ]:
train['DaysPassed'].min(), train['DaysPassed'].max()

In [ ]:
train['num_of_items_per_user'] = train.groupby(['userid'])['itemid'].transform('count')

In [ ]:
train['num_of_items_per_user']=(train['num_of_items_per_user'] - train['num_of_items_per_user'].min())/(train['num_of_items_per_user'] - train['num_of_items_per_user'].min()).max()

In [ ]:
train['ft1'] = train['unixReviewTime'].dt.weekday+train['verified']

In [ ]:
train['ft1']=(train['ft1'] - train['ft1'].min())/(train['ft1'] - train['ft1'].min()).max()

In [ ]:
train.head()

In [ ]:
#Функции для создания списка id для items и users и генерации матриц признаков
def generate_int_id(dataframe, id_col_name):
    new_dataframe=dataframe.assign(
        int_id_col_name=np.arange(len(dataframe))
        ).reset_index(drop=True)
    return new_dataframe.rename(columns={'int_id_col_name': id_col_name})


def create_features(dataframe, features_name, id_col_name):
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col_name], features))
    return features

def generate_feature_list(dataframe, features_name):
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features


In [ ]:
# generating unique ids for users and items
train = generate_int_id(train, 'userid_num')
train = generate_int_id(train, 'itemid_num')

In [ ]:
#list_user_features =['verified','reviewWeekday', ]
list_item_features = ['DaysPassed','num_of_items_per_user','ft1']     


In [ ]:
# creating features for feeding into lightfm 
train['item_features'] = create_features(train, list_item_features, 'itemid_num')

In [ ]:
# generating features list for mapping 
item_feature_list = generate_feature_list(train,list_item_features)

In [ ]:
########################
# Dataset building for lightfm
########################

# define our dataset variable
# then we feed unique professionals and questions ids
# and item and professional feature list
# this will create lightfm internel mapping
dataset = Dataset()
dataset.fit(
    set(train['userid_num']), 
    set(train['itemid_num']),
    user_features=None, 
    item_features=item_feature_list)
    #user_features=user_feature_list

In [ ]:
# creating features for feeding into lightfm 
item_features = dataset.build_item_features(train['item_features'])

#user_features = dataset.build_user_features(df['user_features'])

Разбиваем данные для обучения и обучаем модель:

In [ ]:
# Data preparation for modeling and validation
train_data, test_data = train_test_split(train,random_state=42, shuffle=True)

ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))
sample_weights = sparse.coo_matrix((train_data['weights'], (train_data['userid'],
                                  train_data['itemid'])))

In [ ]:
model = LightFM (learning_rate=0.19121773913492052, 
                loss='logistic',
                learning_schedule= 'adagrad',
                no_components=159)
model = model.fit(ratings_coo, 
 #                 sample_weight = sample_weights, 
                  item_features = item_features,
                  epochs=8,  
                  num_threads=4)


In [ ]:
model.item_biases *= 1.0
preds = model.predict(test_data['userid'].values,
                      test_data['itemid'].values,
                      item_features = item_features)

roc_auc_score(test_data.rating, preds)

Оптимизация параметров алгоритма:

In [ ]:
import itertools

def sample_hyperparameters():
    while True:
        yield {
            "no_components": np.random.randint(10, 200),
            "loss": np.random.choice(["logistic", "bpr","warp"]),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "learning_rate": np.random.exponential(0.05),
            "num_epochs": np.random.randint(5, 50),
        }


def random_search(train, test, num_samples=10, num_threads=1):
    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")

        model = LightFM(**hyperparams)
        model.fit(train, epochs=num_epochs, num_threads=num_threads)
        preds = model.predict(test_data['userid'].values,
                      test_data['itemid'].values)
        score = roc_auc_score(test_data.rating,preds)
        
        hyperparams["num_epochs"] = num_epochs

        yield (score, hyperparams, model)

(score, hyperparams, model) = max(random_search(ratings_coo, test_data, num_samples=50), key=lambda x: x[0])
print(f"Best score {score} at {hyperparams}")


Best score 0.7548034859706412 at {'no_components': 159, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'learning_rate': 0.19121773913492052, 'num_epochs': 8}

In [ ]:
#from sklearn.metrics import roc_auc_score                                                                                                                                                                                                                               
#print(roc_auc_score(test_data.rating,preds)) 

In [ ]:
#print("Train recall: %.2f" % recall_at_k(model, ratings_coo, item_features=item_features, k=5).mean())
#print("Train precision: %.2f" % precision_at_k(model, ratings_coo,item_features=item_features, k=5).mean())

Делаем предсказания *для* сабмишена:

In [ ]:
#готовим тест для предсказания
test['reviewTime'] = pd.to_datetime(test['reviewTime'])
test['unixReviewTime'] = pd.to_datetime(test['unixReviewTime'], unit='s')
# сколько прошло дней
test['DaysPassed'] = (pd.datetime.now() - test['unixReviewTime']).dt.days
test['DaysPassed']=(test['DaysPassed'] - test['DaysPassed'].min())/(test['DaysPassed'] - test['DaysPassed'].min()).max()

test['num_of_items_per_user'] = test.groupby(['userid'])['itemid'].transform('count')
test['num_of_items_per_user']=(test['num_of_items_per_user'] - test['num_of_items_per_user'].min())/(test['num_of_items_per_user'] - test['num_of_items_per_user'].min()).max()

#Конвертируем verified в int
test['verified']=test['verified'].astype('int64')

test['ft1'] = test['unixReviewTime'].dt.weekday+test['verified']
test['ft1']=(test['ft1'] - test['ft1'].min())/(test['ft1'] - test['ft1'].min()).max()

In [ ]:
# generating unique ids for users and items
test = generate_int_id(test, 'userid_num')
test = generate_int_id(test, 'itemid_num')

# creating features for feeding into lightfm 
test['item_features'] = create_features(test, list_item_features, 'itemid_num')

# generating features list for mapping 
item_feature_list_test = generate_feature_list(test,list_item_features)

In [ ]:
#building test dataset with features for prediction
dataset_test = Dataset()
dataset_test.fit(
    set(test['userid_num']), 
    set(test['itemid_num']),
    user_features=None, 
    item_features=item_feature_list_test)

#
item_features_test = dataset_test.build_item_features(test['item_features'])


In [ ]:
preds = model.predict(test.userid.values,
                      test.itemid.values
                      ,item_features=item_features_test
                      )

preds.min(), preds.max()

In [ ]:
#preds = model.predict(test.userid.values,
#                      test.itemid.values)

#preds.min(), preds.max()

In [ ]:
#preds = model.predict(test.userid.values,
#                      test.itemid.values)

#preds.min(), preds.max()

In [ ]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()
normalized_preds.min(), normalized_preds.max()

Сохраняем итоговый файл:

In [ ]:
submission['rating']= normalized_preds
#submission = submission.set_index('Id')
submission.head()

In [ ]:
#0.75227 on lb
submission.to_csv('/content/drive/My Drive/Colab Notebooks/recommendationsv4/submission_lfm_f2.csv')

Сохраняем саму модель:


In [ ]:
model.save('items-recsys_lfm')

Сохранение векторных представлений для дальнейшего деплоя в продакшн:

In [ ]:
item_biases, item_embeddings = model.get_item_representations()

In [ ]:
#Сохранение векторных представлений для модели
import pickle
with open('item_embeddings.pickle', 'wb') as file:
    pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)

Организуем процесс вывода рекомендаций. Для этого воспользуемся методом прохода по графу

In [ ]:
import nmslib
#Создаём наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять наши продукты в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [ ]:
#Вспомогательная функция для поиска по графу для продукта с индексом item_id
def nearest_items_nms(item_id, index, n=3):
    nn = index.knnQuery(item_embeddings[item_id], k=n)
    return nn

In [ ]:
nbm = nearest_items_nms(846,nms_idx)[0]

In [ ]:
#Выводим похожие продукты
test[test.itemid.isin(nbm)]

Пример функции для вывода рекомендаций(доделать recommended):

In [ ]:
#Функция с рекомендациями
def sample_recommendation(model, data, user_ids):
    

    n_users, n_items = data.shape

    for user_id in user_ids:
        #known_positives = data['itemid'][data.tocsr()[user_id].indices]
        known_positives = data['itemid'][(data['rating']==1.0) & (data['userid']==user_id)]
        
        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['itemid'][np.argsort(-scores)]
        
        
        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")
        
        for x in top_items[:3]:
            print("        %s" % x)
        

In [ ]:
sample_recommendation(model, train, [6328, 1137,65])